## Import libraries

In [2]:
import os
import pandas as pd
import ee
from cropengine.models import get_available_models
from cropengine.crop import get_available_crops, get_available_crop_varieties, WOFOSTCropParametersProvider
from cropengine.weather import GEEWeatherDataProvider
from cropengine.soil import IsricSoilDataProvider, WOFOSTSoilParameterProvider
from cropengine.site import WOFOSTSiteParametersProvider
from pcse.base import ParameterProvider

## User inputs

In [11]:
# Location
latitude = 52.53
longitude = 14.12

# Date
start_date = "2020-01-01"
end_date = "2022-12-31"

# Other inputs
# Get the available model info (**To be displayed in the dashboard)
models_info = get_available_models()
model = 'Wofost72_PP' # User can select from 'models_info' dropdown

crops_info = get_available_crops(model)
crop_name = 'wheat' # User can select from 'crops_info' dropdown

crop_varieties_info = get_available_crop_varieties(model, crop_name) # User can select from 'crop_varieties_info' dropdown
crop_variety = 'Winter_wheat_101' # User can select from 'crop_varieties_info' dropdown

## Create workspace directory

In [12]:
workspace_dir = os.path.join('.', 'workspace') # To be decided by Jahid
os.makedirs(workspace_dir, exist_ok=True)

## Data and parameters preparation

### Weather

In [13]:
# ee.Authenticate()
# Prepare weather data for a specific point
meteo_data = GEEWeatherDataProvider(
    start_date=start_date, 
    end_date=end_date, 
    latitude=latitude, 
    longitude=longitude, 
    filepath=os.path.join(workspace_dir, 'meteo.xlsx')
)
meteo_data = meteo_data.save_weather_excel()

GEE not initialized. Attempting initialization...
GEE Initialized successfully.
File saved successfully to ./workspace/meteo.xlsx


### Soil

In [14]:
# Prepare soil data for a specific point
soil_data = IsricSoilDataProvider(
    latitude=latitude, 
    longitude=longitude, 
    depths='0-5cm',
    values=['mean'],
    filepath=os.path.join(workspace_dir, 'soil.csv')
)
# soil_data = soil_data.get_data()
soil_data = pd.read_csv("/beegfs/halder/GITHUB/LEARNING/cropengine/docs/examples/workspace/soil.csv")
soil_data = soil_data[soil_data['metric']=='mean']

# Extract soil parameters from the soil data 
soil_params_provider = WOFOSTSoilParameterProvider(soil_data)
soil_params = soil_params_provider.get_params()

In [16]:
soil_params_info = soil_params_provider.param_metadata # User can change parameters' value from 'soil_params_info' table
soil_params_info = pd.DataFrame(soil_params_info).sort_values(by='parameter').reset_index(drop=True)
print(soil_params_info.shape)
soil_params_info

(46, 4)


,parameter,description,unit,value
0,A0SOM,Initial age of soil organic material,year,None
1,BG_N_SUPPLY,Background supply of N through atmospheric dep...,kg/ha/day,None
2,CNRatioBio,C:N ratio of microbial biomass,kg C kg-1 N,None
3,CNRatioSOMI,Initial C:N ratio of soil organic matter,kg C kg-1 N,8.213
4,CONDfromPF,Table function of the 10-base logarithm of the...,log10(cm water d-1),"[-1.0, 1.672, 1.0, 1.672, 1.3, 1.672, 1.491, 1..."
5,CRAIRC,Critical air content for aeration for root system,m3 air m-3 soil,0.05
6,FASDIS,Fraction of assimilation to dissimilation,-,None
7,FSOMI,Initial fraction of soil organic matter in soil,kg OM kg-1 soil,0.092
8,IFUNRN,Indicates whether non-infiltrating fraction of...,-,None
9,IVINF,Infiltration limiter,-,None


### Site

In [17]:
# Extract site parameters
site_params_provider = WOFOSTSiteParametersProvider(model, WAV=10)
site_params = site_params_provider.get_params()

In [19]:
site_params_info = site_params_provider.param_metadata # User can change parameters' value from 'site_params_info' table
site_params_info = pd.DataFrame(site_params_info).sort_values(by='parameter').reset_index(drop=True)
print(site_params_info.shape)
site_params_info

(6, 7)


,parameter,required,default,range,type,description,unit
0,IFUNRN,False,0.0,"[0, 1]",int,Indicates whether non-infiltrating fraction of...,-
1,NOTINF,False,0.0,"[0.0, 1.0]",float,Maximum fraction of rain not-infiltrating into...,-
2,SMLIM,False,0.4,"[0.0, 1.0]",float,Initial maximum moisture content in initial ro...,-
3,SSI,False,0.0,"[0.0, 100.0]",float,Initial depth of water stored on the surface.,cm
4,SSMAX,False,0.0,"[0.0, 100.0]",float,Maximum depth of water that can be stored on t...,cm
5,WAV,True,NaN,"[0.0, 100.0]",float,Initial amount of water in total soil profile.,cm


### Crop

In [20]:
# Extract crop parameters
crop_params_provider = WOFOSTCropParametersProvider(crop_name=crop_name, variety_name=crop_variety)
print(crop_params_provider)

Crop parameters loaded from: /beegfs/halder/GITHUB/LEARNING/cropengine/cropengine/configs/wofost_crop_params
WOFOSTCropParametersProvider - current active crop 'wheat' with variety 'Winter_wheat_101'
Available crop parameters:
 {'CO2EFFTB': [40.0, 0.0, 360.0, 1.0, 720.0, 1.11, 1000.0, 1.11, 2000.0, 1.11], 'CO2TRATB': [40.0, 0.0, 360.0, 1.0, 720.0, 0.9, 1000.0, 0.9, 2000.0, 0.9], 'CO2AMAXTB': [40.0, 0.0, 360.0, 1.0, 720.0, 1.6, 1000.0, 1.9, 2000.0, 1.9], 'TBASEM': 0.0, 'TEFFMX': 30.0, 'TSUMEM': 120, 'IDSL': 2, 'DLO': 16.8, 'DLC': 8.0, 'TSUM1': 543, 'TSUM2': 1194, 'DTSMTB': [0.0, 0.0, 30.0, 30.0, 45.0, 30.0], 'DVSI': 0.0, 'DVSEND': 2.0, 'VERNBASE': 10.0, 'VERNSAT': 49.0, 'VERNDVS': 0.3, 'VERNRTB': [-8.0, 0.0, -4.0, 0.0, 3.0, 1.0, 10.0, 1.0, 17.0, 0.0, 20.0, 0.0], 'TDWI': 50.0, 'RGRLAI': 0.0082, 'SLATB': [0.0, 0.00212, 0.5, 0.00212, 2.0, 0.00212], 'SPA': 0.0, 'SSATB': [0.0, 0.0, 2.0, 0.0], 'SPAN': 31.3, 'TBASE': 0.0, 'KDIFTB': [0.0, 0.6, 2.0, 0.6], 'EFFTB': [0.0, 0.45, 40.0, 0.45], 'AMAXT

In [21]:
crop_params_info = crop_params_provider.param_metadata # User can change parameters' value from 'crop_params_info' table
crop_params_info = pd.DataFrame(crop_params_info).sort_values(by='parameter').reset_index(drop=True)
print(crop_params_info.shape)
crop_params_info

(94, 4)


,parameter,description,unit,default
0,AMAXTB,maximum leaf CO2 assimilation rate as function...,"[-, kg.ha-1.hr-1]","[0.0, 35.83, 1.0, 35.83, 1.3, 35.83, 2.0, 4.48]"
1,CFET,correction factor transpiration rate,-,1.0
2,CO2AMAXTB,multiplication factor for AMAX to account for ...,"[PPM, -]","[40.0, 0.0, 360.0, 1.0, 720.0, 1.6, 1000.0, 1...."
3,CO2EFFTB,multiplication factor for EFF to account for a...,"[PPM, -]","[40.0, 0.0, 360.0, 1.0, 720.0, 1.11, 1000.0, 1..."
4,CO2TRATB,multiplication factor for maximum transpiratio...,"[PPM, -]","[40.0, 0.0, 360.0, 1.0, 720.0, 0.9, 1000.0, 0...."
...,...,...,...,...
89,TSUMEM,temperature sum from sowing to emergence,C.d,120
90,VERNBASE,Base vernalization requirement,d,10.0
91,VERNDVS,Critical DVS for vernalization to switch off,-,0.3
92,VERNRTB,Temperature response function for vernalization,"[C, -]","[-8.0, 0.0, -4.0, 0.0, 3.0, 1.0, 10.0, 1.0, 17..."


### Packing all parameters

In [22]:
crop_dict = dict(zip(crop_params_info['parameter'], crop_params_info['default']))
soil_params_info = soil_params_info.dropna()
soil_dict = dict(zip(soil_params_info['parameter'], soil_params_info['value']))
site_dict = site_params.copy()

parameters = ParameterProvider(cropdata=crop_dict, soildata=soil_dict, sitedata=site_dict)

## Define agromanagement

In [ ]:
# -----------------------------------------------------------------------------
# User Configuration Implementation
# -----------------------------------------------------------------------------

def generate_agromanagement_config(
    start_date_str: str,
    end_date_str: str,
    crop_name: str,
    crop_variety: str
):
    # Convert strings to dates
    sim_start_date = datetime.datetime.strptime(start_date_str, "%Y-%m-%d").date()
    sim_end_date = datetime.datetime.strptime(end_date_str, "%Y-%m-%d").date()
    
    manager = DashboardAgroManagementProvider()

    # --- Strategy for Winter Wheat ---
    # Winter wheat is sown in Autumn (e.g., Oct 15th).
    # We iterate through the years to find valid sowing windows within the user's range.
    
    current_year = sim_start_date.year
    end_year = sim_end_date.year
    
    # Track the last campaign start to ensure valid spacing
    last_campaign_start = sim_start_date

    for year in range(current_year, end_year + 1):
        
        # Define Sowing Date: October 15th of the current year
        sowing_date = datetime.date(year, 10, 15)
        
        # Check if this sowing date fits within our simulation window
        # We also need to ensure the crop has time to grow (e.g. +200 days) before sim_end_date
        maturity_buffer = datetime.timedelta(days=250)
        
        if sowing_date >= sim_start_date and (sowing_date + maturity_buffer) <= sim_end_date:
            
            # The campaign usually starts shortly before sowing or at the start of the gap
            # For the first year, we align with simulation start.
            # For subsequent years, we start the campaign on Sept 1st (pre-sowing).
            if year == current_year:
                campaign_start = sim_start_date
            else:
                campaign_start = datetime.date(year, 9, 1)

            manager.add_campaign(
                campaign_start_date=campaign_start,
                crop_name=crop_name,
                variety_name=crop_variety,
                crop_start_date=sowing_date,
                crop_start_type="sowing",
                crop_end_type="maturity", # Let model determine harvest based on phenology
                max_duration=300,         # Winter wheat needs a long duration
                timed_events=None,        # No irrigation for Potential Production (PP)
                state_events=None
            )
            last_campaign_start = campaign_start

    # Add a trailing empty campaign to force simulation to run until the user's defined end_date
    # This covers the period after the last harvest until Dec 31st 2022
    manager.add_trailing_empty_campaign(sim_end_date)

    return manager

# -----------------------------------------------------------------------------
# Execution
# -----------------------------------------------------------------------------

# Inputs from Dashboard
inputs = {
    "start_date": "2020-01-01",
    "end_date": "2022-12-31",
    "crop_name": "wheat",
    "crop_variety": "Winter_wheat_101"
}

# Generate the PCSE-compliant object
agromanagement = generate_agromanagement_config(
    inputs["start_date"],
    inputs["end_date"],
    inputs["crop_name"],
    inputs["crop_variety"]
)

# For verification: Print the structure
print(f"Generated {len(agromanagement)} campaign(s) + trailing end.")
for campaign in agromanagement:
    date_key = list(campaign.keys())[0]
    data = campaign[date_key]
    print(f"\nCampaign Start: {date_key}")
    if data:
        print(f"  Crop: {data['CropCalendar']['crop_name']}")
        print(f"  Sowing: {data['CropCalendar']['crop_start_date']}")
    else:
        print("  (Trailing empty campaign to finalize simulation)")